In [16]:
from typing import Annotated, Optional, List, Literal, Any
from langchain_core.tools import tool
from typing_extensions import TypedDict
from datetime import datetime

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from pydantic import BaseModel, Field, EmailStr, HttpUrl, RootModel
import requests
from rich import print_json
from dotenv import load_dotenv
import os
import json
from langgraph.types import Command, interrupt
from IPython.display import Image, display
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langfuse import Langfuse
from langfuse.callback import CallbackHandler
from concurrent.futures import ThreadPoolExecutor
import asyncio

load_dotenv()

devskiller_api_url = "https://api.devskiller.com"

In [35]:
class Link(BaseModel):
    href: HttpUrl


class Links(BaseModel):
    candidate: Link
    assessment: Link
    invitation: Link


class EventPayloadItem(BaseModel):
    candidateId: str
    candidateExternalId: str
    assessmentId: str
    candidateEmail: EmailStr
    examId: str
    testId: str
    examName: str
    event: str
    _links: Links


class EventPayload(RootModel[List[EventPayloadItem]]):
    pass

event_payload = [
  {
    "candidateId": "fbd1b0af-25e1-4576-a078-c5c8b6659974",
    "candidateExternalId": "129d0fae-ba28-4d5d-960c-64c3b570d174",
    "assessmentId": "11e98fec-dc81-4677-a9cc-5b8b7e9f816c",
    "candidateEmail": "ivor.padilla@gmail.com",
    "examId": "8b1e4189-0017-420e-b022-dcdc72f51be4",
    "testId": "8b1e4189-0017-420e-b022-dcdc72f51be4",
    "examName": "Frontend RLHF Test C",
    "event": "WAITING_FOR_ASSESSMENT",
    "_links": {
      "candidate": {
        "href": "https://api.devskiller.com/candidates/fbd1b0af-25e1-4576-a078-c5c8b6659974"
      },
      "assessment": {
        "href": "https://api.devskiller.com/candidates/fbd1b0af-25e1-4576-a078-c5c8b6659974/assessments/11e98fec-dc81-4677-a9cc-5b8b7e9f816c"
      },
      "invitation": {
        "href": "https://api.devskiller.com/invitations/11e98fec-dc81-4677-a9cc-5b8b7e9f816c"
      }
    }
  }
]

headers = { 
    "Devskiller-Api-Key": os.getenv("DEVSKILLER_API_KEY"), 
    "Content-Type": "application/vnd.devskiller.v2.hal+json", 
    "Accept": "application/vnd.devskiller.v2.hal+json" 
}

langfuse_handler = CallbackHandler(
  secret_key="sk-lf-bd57965d-e7d4-4647-acc7-5a3915e5d662",
  public_key="pk-lf-8e0cd992-220c-46ed-8339-cd6552a2f63d",
  host="https://cloud.langfuse.com"
)

class State(BaseModel):
    payload: Any
    assessment: Optional[Any] = {}

In [36]:
async def retrieve_devskiller_assessment(state: State):
    event = state.payload
    event = EventPayload.model_validate(event).root[0]
    candidate_id, assessment_id = event.candidateId, event.assessmentId
    url = f"{devskiller_api_url}/candidates/{candidate_id}/assessments/{assessment_id}"
    print(json.dumps(headers, sort_keys=True, indent=4))
    response = requests.get(url, headers=headers)
    return { "assessment": response.json() }

In [37]:
workflow = StateGraph(State)
workflow.add_node("retrieve_devskiller_assessment", retrieve_devskiller_assessment)
workflow.add_edge(START, "retrieve_devskiller_assessment")
workflow.add_edge("retrieve_devskiller_assessment", END)
graph = workflow.compile()

config = {"configurable": {"callbacks": [langfuse_handler]}}

In [38]:
await graph.ainvoke(State(payload=event_payload), config=config)

{
    "Accept": "application/vnd.devskiller.v2.hal+json",
    "Content-Type": "application/vnd.devskiller.v2.hal+json",
    "Devskiller-Api-Key": "5e96d1b9-2b8e-4d81-8eef-af959466c8f8"
}


{'payload': [{'candidateId': 'fbd1b0af-25e1-4576-a078-c5c8b6659974',
   'candidateExternalId': '129d0fae-ba28-4d5d-960c-64c3b570d174',
   'assessmentId': '11e98fec-dc81-4677-a9cc-5b8b7e9f816c',
   'candidateEmail': 'ivor.padilla@gmail.com',
   'examId': '8b1e4189-0017-420e-b022-dcdc72f51be4',
   'testId': '8b1e4189-0017-420e-b022-dcdc72f51be4',
   'examName': 'Frontend RLHF Test C',
   'event': 'WAITING_FOR_ASSESSMENT',
   '_links': {'candidate': {'href': 'https://api.devskiller.com/candidates/fbd1b0af-25e1-4576-a078-c5c8b6659974'},
    'assessment': {'href': 'https://api.devskiller.com/candidates/fbd1b0af-25e1-4576-a078-c5c8b6659974/assessments/11e98fec-dc81-4677-a9cc-5b8b7e9f816c'},
    'invitation': {'href': 'https://api.devskiller.com/invitations/11e98fec-dc81-4677-a9cc-5b8b7e9f816c'}}}],
 'assessment': {'id': '11e98fec-dc81-4677-a9cc-5b8b7e9f816c',
  'status': 'AUTO_ASSESSMENT_READY',
  'timeTakenInSeconds': 417,
  'timeLimitInSeconds': 661,
  'score': {'scoredPoints': 0, 'maxPoin